In [2]:
import gzip
import torch 
import torchvision
import numpy as np
import pandas as pd
from torch import nn
import torchkeras
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split

torchkeras.LightModel can't be used!


In [19]:
class RNN(nn.Module): 
    def __init__(self, input_size, output_size, layer_num, hidden_size, activation) -> None:
        super().__init__() 
        if activation == 'R': self.activation = nn.ReLU() 
        if activation == 'T': self.activation = nn.Tanh() 
        if activation == 'S': self.activation = nn.Sigmoid() 
        
        self.layer_num = layer_num
        self.hidden_size = hidden_size
        self.input_size = input_size
        def normal(shape):
            import math
            return (torch.rand(shape) - 0.5) * 1.0 / math.sqrt(150)
        self.W_hx = nn.Parameter(normal((self.layer_num, self.input_size, self.hidden_size)))  
        self.b_hx = nn.Parameter(normal((self.layer_num, self.hidden_size)))
        self.W_hh = nn.Parameter(normal((self.layer_num, self.hidden_size, self.hidden_size))) 
        self.b_hh = nn.Parameter(normal((self.layer_num, self.hidden_size)))
        # self.W_ho = {i : nn.Linear(self.hidden_size, self.hidden_size) for i in range(0, self.layer_num)}
        self.O = nn.Linear(hidden_size, output_size) 
        
        # for x in self.W_hx.values(): 
        # self.reset_parameters(self.W_hx) 
        # for x in self.W_hh.values(): 
            # self.reset_parameters(x) 
        # for x in self.W_ho.values(): 
            # self.reset_parameters(x) 
        # self.reset_parameters(self.O) 

    
    def reset_parameters(self, x):
        # print(x)
        import math
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in x.parameters():
            torch.nn.init.uniform_(weight, -stdv, stdv)
            # weight = nn.Parameter(weight)
            # print(weight.requires_grad)
            # print(weight)


    def forward(self, X): 
        X = {i : X[:, :, i] for i in range(self.layer_num)}
        H = {}; O = {}
        H[0] = torch.zeros(self.hidden_size)#, requires_grad=True) # 

        for i in range(0, self.layer_num): 
            # if i == 14:
                # print(self.W_hh[i](H[i]).shape, self.W_hx[i](X[i])[13], self.W_hx[i](X[i])[17])
            H[i + 1] = self.activation(H[i] @ self.W_hh[i] + self.b_hh[i] + X[i] @ self.W_hx[i] + self.b_hx[i]) 
            # O[i] = self.activation(self.W_ho[i](H[i]))
        # print(H[3][1], H[3][2],H[3].shape)
        # print(self.O(O[self.layer_num - 1])[:5]) 
        return self.O(H[self.layer_num]).squeeze() 

R = RNN(28, 10, 28, 128, 'R') 
x = torch.rand(64, 28, 28) 
R(x).shape

torch.Size([64, 10])

In [4]:
import math
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l
batch_size, num_steps = 32, 35
# train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)


# F.one_hot(torch.tensor([0, 2]), len(vocab))
X = torch.arange(10).reshape((2, 5))
F.one_hot(X.T, 28).shape

def get_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size

    def normal(shape):
        return torch.randn(size=shape, device=device) * 0.01

    W_xh = normal((num_inputs, num_hiddens))
    W_hh = normal((num_hiddens, num_hiddens))
    b_h = torch.zeros(num_hiddens, device=device)
    W_hq = normal((num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs, device=device)
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device),)

def rnn(inputs, state, params):
    print("wa")
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    print(inputs.shape, "haha")
    for X in inputs:
        H = torch.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h)
        Y = torch.mm(H, W_hq) + b_q
        outputs.append(Y)
    return torch.cat(outputs, dim=0), (H,)

class RNNModelScratch(nn.Module):  
    """从零开始实现的循环神经网络模型"""
    def __init__(self, vocab_size, num_hiddens, device, get_params,
                 init_state, forward_fn):
        super().__init__()
        print("haha")
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        self.params = get_params(vocab_size, num_hiddens, device)
        self.init_state, self.forward_fn = init_state, forward_fn

    def __call__(self, X, state):
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        print("wqe")
        return self.forward_fn(X, state, self.params)

    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)

# num_hiddens = 512
# net = RNNModelScratch(28, 150, d2l.try_gpu(), get_params,
#                       init_rnn_state, rnn)
# state = net.begin_state(X.shape[0], d2l.try_gpu())
# Y, new_state = net(X.to(d2l.try_gpu()), state)
# Y.shape, len(new_state), new_state[0].shape

In [5]:
class nn_RNN(nn.Module): 
    def __init__(self, input_size, output_size, layer_num, hidden_size, activation):
        super(nn_RNN,self).__init__()

        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=1,
            batch_first=True,
        )
        self.out = nn.Linear(hidden_size,output_size)

    def forward(self, x):
        # print(x.shape)
        x = x.squeeze().permute(2, 0, 1)
        r_out,h_n = self.rnn(x,None)
        # print(r_out.shape)

        out = self.out(r_out[-1 ,:,:])

        # print(out.shape)
        return out

        # b * 28 * 28

In [6]:
class o_RNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.rnn = nn.RNN(
            input_size=28,
            hidden_size=64,
            num_layers=1,
            batch_first=True,
        )
        self.out = nn.Linear(64,10)

    def forward(self, x):
        x = x.reshape(-1, 28, 28)
        r_out,h_n = self.rnn(x,None)
        out = self.out(h_n[-1])
        return out